# 2. Docker

- [강의 자료 링크 (pdf)](https://drive.google.com/file/d/1yffWrQgYCNVZPVRmqMXZiHwu7aPpLqPf/view?usp=sharing)

<br>

## 2.1 주요 명령어

### 2.1.1 설치 확인


`docker`

- 도커 설치 확인

<br>

### 2.1.2 도커 이미지 pull

`docker pull --platform linux/amd64 이미지이름:태그`

- 도커 이미지 다운로드
- m1 mac은 `--platform linux/amd64`를 추가해줘야 한다.
- ex) mysql 8 버전 설치
  - `docker pull --platform linux/amd64 mysql:8`

<br>

### 2.1.3 이미지 목록 확인

`docker images`

- 다운로드 받은 이미지 확인

<br>

### 2.1.4 이미지 기반 도커 컨테이너 생성 및 실행

`docker run "이미지 이름:태그"`

- 다운로드 받은 이미지 기반으로 docker container 만들고 실행
- ex) MySQL 이미지 기반 docker container 실행

```
docker run --name mysql-tutorial -e MYSQL_ROOT_PASSWORD=1234 -d -p 3306:3306 mysql:8
```

- `--name`
  - 컨테이너 이름 지정
  - 지정하지 않으면 랜덤으로 생성
- `-e`
  - 환경변수 설정
  - 사용하는 이미지에 따라 설정이 다름
- `-d`
  - 데몬(백그라운드) 모드
  - 컨테이너를 백그라운드 형태로 실행
  - 이 설정을 하지 않으면, 현재 실행하는 shell 위에서 컨테이너가 실행됨
  - 그렇게 되면 컨테이너의 로그를 바로 볼 수 있으나, 컨테이너를 나가면 실행 종료
- `-p`
  - 포트 정보
  - `로컬 호스트 포트:컨테이터 포트`
  - 로컬 포트 3306으로 접근 시 컨테이너 포트 3306으로 연결되도록 설정
  - 로컬 호스트: 우리의 컴퓨터
  - 컨테이너: 컨테이너 이미지 내부
- `-v`
  - volume mount
  - 로컬 저장소와 도커 컨테이너 내부의 저장소 공유
  - `호스트 폴더:컨테이터 폴더`

<br>

### 2.1.4 실행중인 컨테이너 확인

`docker ps`

- 실행 중인 도커 컨테이너 확인

`docker ps -a`

- 작동을 멈춘 컨테이너 목록 확인

<br>

### 2.1.5 컨테이너 진입

`docker exec -it "컨테이너 이름(or ID)" /bin/bash`

- 컨테이너 진입
- SSH 접속과 유사

<br>

### 2.1.6 컨테이너 중지

`docker stop "컨테이너 이름(or ID)"`

- 실행중인 컨테이너 중지

<br>

### 2.1.7 컨테이너 삭제

`docker rm "컨테이너 이름(or ID)"`

- 멈춘 도커 컨테이너 삭제
- 멈춘 컨테이너만 삭제할 수 있지만 `docker rm "컨테이너 이름(or ID)" -f` 로 실행 중인 컨테이너도 삭제 가능

<br>

## 2.2 docker 파일 공유

- `docker run` 할 때 `-v` 옵션을 사용하여 host와 container의 폴더를 공유할 수 있다.
  - `-v host_folder:container_folder`
- ex) 주피터 노트북 컨테이너 사용 시
  - `docker run -it -p 8888:8888 -v /some/host/folder/for/work:/home/jovyan/workspace/jupyter/minimal-notebook`

<br>

## 2.3 docker image 만들기

### 2.3.1 프로젝트 세팅

```
python -m venv .venv
source .venv/bin/activate
pip install pip --upgrade
pip install "fastapi[all]"
```

<br>

### 2.3.2 FastAPI 코드 작성

`main.py`

```python
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/hello")
def hello():
    return {"message": "World!"}


if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

<br>

### 2.3.3 사용한 라이브러리 명시

- `pip freeze`
  - 설치한 라이브러리 확인
- `pip list --not-required --format=freeze`
  - 설치한 라이브러리 확인
  - 의존성에 따라 설치된 라이브러리는 보이지 않음
- `pip freeze > requirements.txt`
  - pip로 설치한 라이브러리 모두 `requirements.txt`에 저장

<br>

### 2.3.4 Dockerfile 작성

`Dockerfile`

- Dockerfile에는 도커 이미지를 만들기 위한 정보들을 담고 있다.

```docker
FROM python:3.8.7-slim-buster

COPY . /app
WORKDIR /app
ENV PYTHONPATH=/app
ENV PYTHONBUFFERED=1

RUN pip install pip==21.2.4 && \
    pip install pip --upgrade && \
    pip install -r requirements.txt

CMD ["python", "main.py"]
```

<br>

`FROM 이미지 이름:태그`

- 이미지 빌드에 사용할 베이스 이미지 지정
- 베이스 이미지는 이미 만들어진 이미지
- 보통 처음부터 만들지 않고, 이미 공개된 이미지를 기반으로 새로운 설정을 추가
- `FROM python:3.8.7-slim-buster`
  - Dockerhub에 존재하는 `python:3.8.7-slim-buster` 라는 이미지를 사용

<br>

`COPY "로컬 디렉토리(파일)" "컨테이너 내 디렉토리(파일)"`

- 컨테이너는 자체적인 파일 시스템과 디렉토리를 가짐
- `COPY` 명령어는 Dockerfile이 존재하는 경로 기준 로컬 디렉토리를 컨테이너 내부의(자체 파일 시스템을 가진) 디렉토리로 복사
- 파일을 컨테이너에서 사용하려면 `COPY` 명령어로 반드시 복사해야 함
- `COPY . /app`
  - 프로젝트 최상위에 존재하는 모든 파일을 컨테이너 내부 `/app` 디렉토리로 복사

<br>

`WORKDIR "컨테이너 내 디렉토리"`

- Dockerfile의 `RUN`, `CMD`, `ENTRYPOINT` 등의 명령어를 실행할 컨테이너 경로 지정
- `WORKDIR /app`
  - 이 라인 뒤에 등장하는 `RUN`, `CMD`는 컨테이너 내부의 `/app`에서 실행

<br>

`ENV "환경변수 이름=값"`

- 컨테이너 내 환경변수를 지정
- 파이선 애플리케이션의 경우 통상 다음 두 값을 지정
  - `ENV PYTHONPATH=/app`
  - `ENV PYTHONBUFFERED=1`

<br>

`RUN "실행할 리눅스 명령어"`

- 컨테이너 내에서 리눅스 명령어를 실행
- 실행해야 할 명령어가 여러 개인 경우 `&& \`로 이어줌
  - `RUN` 명령어를 하나의 레이어로 묶어주는 게 좋기 때문에 `RUN`을 여러 번 쓰는 것보다 명령어들을 묶어주는 것이 좋다.
- 이전 라인에서 `COPY`와 `WORKDIR`이 실행되었기 때문에 컨테이너 내에 `requirements.txt`가 존재하고 이를 `pip install -r` 명령어로 실행시킬 수 있음

<br>

`CMD ["실행할 명령어", "인자", ...]`

- `docker run`으로 이 이미지를 기반으로 컨테이너를 만들 때, 실행할 명령어
- `CMD ["python", "main.py"]`
  - 이 이미지는 실행되는 즉시 `python main.py`를 실행한다.
- `CMD`는 띄어쓰기를 사용하지 않음

<br>

`RUN` vs `CMD` vs `ENTRYPOINT`

- [https://blog.leocat.kr/notes/2017/01/08/docker-run-vs-cmd-vs-entrypoint](https://blog.leocat.kr/notes/2017/01/08/docker-run-vs-cmd-vs-entrypoint)
- 처음에는 3개 중 하나를 사용하고 점차 확장해나가는 것을 추천

<br>

`EXPOSE`

- 컨테이너 외부에 노출할 포트 지정

<br>

`ENTRYPOINT`

- 이미지를 컨테이너로 띄울 때 항상 실행하는 커맨드

<br>

### 2.3.5 Docker Image Build

- `docker build "Dockerfile이 위치한 경로" -t "이미지 이름:태그"`
- `docker build . -t my-fastapi-app --platform linux/amd64`

<br>

### 2.3.6 컨테이너 실행

- `docker run -p 8000:8000 my-fastapi-app`

<br>

## 2.4 Registry에 Docker Image Push

- 만든 이미지를 인터넷에 업로드
- 이를 위해 이미지 저장소인 **Container Registry**에 Docker Image Push
- Container Registry
  - Dockerhub, GCP GCR, AWS ECR 등
- 보통 어떤 클라우드 서비스로 배포할 지에 따라 어떤 레지스트리 서비스를 사용할 지 결정
  - ex) GCP에서 배포한다면, 레지스트리도 역시 GCP 서비스인 GCR을 사용하는 식
- 별도로 지정하지 않으면 기본적으로 Dockerhub을 사용
- 우리는 GCP의 GCR을 사용

<br>

### 2.4.1 GCR 설정

- Container Registry

<br>

### 2.4.2 gcloud 설정

- gcloud 설치
- [Cloud SDK Install](https://cloud.google.com/sdk/docs/install)

```
~/program/google-cloud-sdk/install.sh
```

- `gcloud auth login`

- `gcloud config set project "프로젝트 ID"`
- `gcloud config set project product-serving`

- `gcloud auth configure-docker`
  - Docker 설정

<br>

### 2.4.3 Tag 설정

- `docker tag "기존 이미지:태그" "새 이미지 이름:태그"`
  - gcr에 올릴 이미지 이름은 `gcr.io/GCP "프로젝트 이름/이미지 이름"` 형태여야 한다.
- `docker tag my-fastapi-app gcr.io/product-serving/my-fastapi-app`

<br>

### 2.4.4 Push

- `docker push "이미지 이름:태그"`
- `docker push gcr.io/product-serving/my-fastapi-app`

<br>

## 2.5 Registry의 Docker Image Pull

- `docker pull gcr.io/product-serving/my-fastapi-app`

<br>

## 2.6 Docker 이미지로 배포하기 (1) - Cloud Run

### 2.6.1 Serverless Cloud 서비스 - Cloud Run

- 도커 이미지를 서버에 배포하는 가장 간단한 방법
  - Cloud 서비스 활용
  - GCP: Cloud Run
  - AWS: ECS

<br>

## 2.7 Docker 이미지로 배포하기 (2) - Compute Engine

다른 방식으로 Docker Image 배포
- Compute Engine을 띄우고, 해당 인스턴스 실행될 때 Docker Image를 가지고 실행하도록 설정
- Github Action을 사용해 Docker Image Push 자동화!
- Part 2 - CI/CD에서 진행한 Streamlit 파일을 기반으로 실행

<br>

### 2.7.1 환경 설정

GCP에서 서비스 계정 생성

- IAM 및 관리자 - 서비스 계정
- IAM(Identity and Access Management)
  - 클라우드 서비스에 접근 권한을 관리하는 서비스
- 서비스 계정(Service Account)
  - 사용자가 아닌 애플리케이션, VM 인스턴스에서 사용하는 임의의 계정
  - 프로젝트의 관리자가 서비스 계정에 권한을 부여할 수 있음

<br>

서비스 계정 key 생성

- json 파일 형태의 key가 생성됨
- 이 key가 노출되면 자신의 프로젝트가 해킹될 수 있으니 공유 금지

<br>

IAM에 생성한 서비스 계정 권한 부여

<br>

Github Repository의 Secret 추가

- `SERVICE_ACCOUNT_KEY`
  - 위에서 생성한 서비스 계정의 key json 파일 내용
- `GCP_PROJECT_ID`
  - gcp 프로젝트 ID
- `GCE_INSTANCE`
  - 인스턴스 이름
- `GCE_INSTANCE_ZONE`
  - 인스턴스의 zone

<br>

### 2.7.2 GCR Push

`Dockerfile` 생성

<br>

`copy_asset.sh` 실행

<br>

로컬에서 도커 이미지 빌드

`docker build . -t "streamlit" --platform linux/amd64`

<br>

도커 이미지 생성 확인

`docker images | grep "streamlit"`

<br>

태그(tag) 설정

`docker tag streamlit gcr.io/product-serving/streamlit`

<br>

GCR에 push

`docker push gcr.io/product-serving/streamlit`

- 시간이 많이 소요된다.
- 왜 이렇게 느릴까?
- 더 빠르게 하려면 어떻게 해야할까?
- 현재 오래 걸리는 구간은 어디인가?
  - 현재 4GB 용량을 차지하는데 이 이미지를 더 작은 것으로 변경하면 속도가 개선될 것이다.
  - efficientnet pytorch 설치가 오래 소요 -> "무거운 라이브러리가 있는 경우 이슈가 있구나"를 인지

<br>

GCR에서 Docker Image 확인

<br>

### 2.7.3 수동 실행

Compute Engine 인스턴스 생성

- 부팅 디스크 용량은 50GB로
- 컨테이너 구성
  - 아까 GCR에 push한 컨테이너 이미지 지정
  - 최신 이미지를 사용
- 네크워크 태그 추가
  - `streamlit`
- 인스턴스가 생성되면 외부 IP로 접근

<br>

### 2.7.4 문제 발생 시

만약 IP 주소에 접근할 수 없는 경우, 원인 파악 하나씩! (실무에서도 문제는 매번 발생)

- 방화벽 설정 잘 되어 있는가?
- 도커 이미지가 잘 push 되어 있는가?
- Compute Engine 실행하면서 Docker를 잘 실행했는가?
  - 확인하기 위해 SSH로 인스턴스 접속
  - `docker images`를 진행하고 우리의 GCR Image 있는 지 확인
  - 없다면 컨테이너 실행 로그 확인
  - Compute Engine에 SSH 접속 후 다음 명령어를 입력하면 Container 실행하는 로그가 나옴
    - `sudo journalctl -u konlet-startup`
  - 로그 메시지 확인하며 다시 문제 해결
  
    

<br>

### 2.7.5 Github Action

Github Action Workflow 생성하기

- 작업 흐름
  1. Feature/xxx Branch에서 작업
  2. Main Branch로 Pull Request
  3. Review 후 Merge
  4. Merge된 파일에서 Docker Image Build
  5. Docker Image Push
  6. Compute Engine에 Docker Image 업데이트 요청

<br>

## 2.8 Docker Compose

- 하나의 Docker Image가 아니라 여러 Docker Image를 동시에 실행하고 싶다면?
- 혹은 A Image로 Container를 띄우고, 그 이후에 B Container를 실행해야 하는 경우
  - ex) A는 Database이고, B는 웹 서비스인 경우
- `docker run` 할 때 옵션이 너무 다양하고, Volume Mount를 하지 않았다면 데이터가 모두 날라감

- 이럴 경우 활용할 수 있는 것이 Docker Compose이다.
  - **여러 컨테이너를 한 번에 실행할 수 있음**
  - 여러 컨테이너의 실행 순서, 의존도를 관리할 수 있음
  - `docker-compose.yml` 파일에 작성

<br>

### 2.8.1 `docker-compose.yml`

- docker-compose는 다음처럼 `docker-compose.yml` 파일에 특정 문법으로 작성
- 다음 코드는 db 컨테이너와 app 컨테이너 두 개를 실행시키는 내용

```
version: '3'

services:
  db:
    image: mysql:5.7.12
    environment:
      MYSQL_ROOT_PASSWORD: root
      MYSQL_DATABASE: my_database
    ports:
      - 3306:3306

  app:
    build:
      context: .
    environment:
      DB_URL: mysql+mysqldb://root:root@db:3306/my_database?charset=utf8mb4
    ports:
      - 8000:8000
    depends_on:
      - db
    restart: always
```

<br>

### 2.8.2 `docker-compose up`

- Docker Image 일괄 실행
- `docker-compose.yml` 파일을 파싱하여 container 실행
- 이 때 필요한 이미지를 pull 하거나 build 하는 등의 과정도 포함

<br>

### 2.8.3 `docker-comose`의 다양한 명령어들

- `docker-compose up -d`
  - 백그라운드에서 실행하기 (`docker run -d`와 동일)
- `docker-compose down`
  - 서비스 중단 (컨테이너, 볼륨 등 삭제)
- `docker-compose logs <서비스명>`
  - 로그 확인

- 참고로 `docker-compose.yml` 파일을 수정하고 `docker-compose up`을 하면 **컨테이너를 재생성하고, 서비스를 재시작함**

<br>

### 2.8.4 실행 중인 컨테이너 확인

- `docker-compose up`이 완료되면 다음처럼 `docker ps` 명령어나 `docker-compose ps` 명령어로 현재 실행되고 있는 컨테이너를 확인할 수 있음

<br>

## 2.9 Special Mission

1. Docker 설치해서 MySQL 설치하기
2. Docker를 사용해 Jupyter Notebook 설치하기
3. Jupyter Notebook Compute Engine에 배포하기
4. Voila, Streamlit에서 진행한 내용 Docker Image로 말아서 배포하기
5. Docker Compose로 Jupyter Notebook 설치하기